In [19]:
suppressPackageStartupMessages({
    library(data.table)
    library(tidyverse)
})

In [20]:
config <- list(
    seed = 1923,
    folds = 10,
    learning_rate = 0.01,
    iter = 1000,
    loss_fun = 'F1:use_weights=False',
    eval_metric = 'F1:use_weights=False',
    task = 'GPU'
)

In [21]:
sparse_data <- function(df,sparse_percentage = 0.02){
    df_new <- data.table::copy(df) 
    setDT(df_new)
    categorical_names <- df_new %>% purrr::discard(is.numeric) %>% colnames

    for(i in categorical_names){
        features <- df_new[,unique(.SD),.SDcols = i] %>% pull
        
        for(j in features){
            
            condition_format <- 'df_new[,length(%s[%s == "%s"])/.N < %s]'
            condition_command <- sprintf(condition_format,i,i,j,sparse_percentage)
        
            condition <- eval(parse(text = condition_command))
            
            if(condition){
                sparse_format <- 'df_new[%s == "%s", %s := "other"]'
                sparse_command <- sprintf(sparse_format,i,j,i)
                eval(parse(text = sparse_command))
            } 
        }
    }
    df_new
}

In [26]:
clean_test_set <- function(df,categorical_cols,distinct_values_on_train){
    df_new <- copy(df)
    
    make_paste <- function(x){
    wrapped <- sapply(x,function(x) paste0('"',x,'"'))
    paste0(wrapped,collapse = ',')
    }       
   
    
    for(i in categorical_cols){
        distincts <- distinct_values_on_train[[i]]
        distincts <- make_paste(distincts)
        command_format <- "df_new[! %s %%in%% c(%s), %s := 'Missing']"
        command <- sprintf(command_format,i,distincts,i)
        eval(parse(text = command))
    }                   
    df_new
}